<a href="https://colab.research.google.com/github/ccstevie/nhl_model/blob/main/NHL_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
from datetime import date, timedelta
from functools import reduce

In [2]:
# we will use the last 30 days as our data range
today = date.today()
print("Today's date:", today)
start = today - timedelta(days=30)
print("30 days ago:", start)

Today's date: 2024-03-04
30 days ago: 2024-02-03


In [3]:
url = f"https://www.naturalstattrick.com/teamtable.php?fromseason=20232024&thruseason=20232024&stype=2&sit=5v5&score=all&rate=n&team=all&loc=B&gpf=410&fd={start}&td={today}"
req = requests.get(url)
req.status_code

200

In [4]:
df = pd.read_html(url, header=0, index_col = 0, na_values=["-"])[0]
df.head()

,Team,GP,TOI,W,L,OTL,ROW,Points,Point %,CF,...,LDSA,LDSF%,LDGF,LDGA,LDGF%,LDSH%,LDSV%,SH%,SV%,PDO
1,Colorado Avalanche,13,638:44,5,6,2,5,12,0.462,633,...,158,48.87,5,5,50.00,3.31,96.84,7.79,92.65,1.004
2,New York Rangers,12,569:51,10,1,1,10,21,0.875,540,...,140,48.34,10,2,83.33,7.63,98.57,9.06,94.91,1.040
3,New York Islanders,11,543:13,6,3,2,6,14,0.636,482,...,117,50.63,4,6,40.00,3.33,94.87,9.60,92.05,1.017
4,Toronto Maple Leafs,13,658:41,10,3,0,9,20,0.769,673,...,124,52.67,5,3,62.50,3.62,97.58,10.65,90.41,1.011
5,Calgary Flames,11,525:47,8,3,0,8,16,0.727,574,...,118,51.64,5,4,55.56,3.97,96.61,9.89,91.83,1.017


In [5]:
# sort by CF% GF% XGF% HDCF% SH% and obtain rankings for each team
cf = df.sort_values(by="CF%", ascending=False, ignore_index=True)
cf.index += 1
cf.reset_index(inplace=True)
cf = cf.rename(columns = {"index":"CF%"})
cf = cf[["Team", "CF%"]]
# print cf as an example
cf

,Team,CF%,CF%
0,Carolina Hurricanes,1,57.94
1,Edmonton Oilers,2,56.89
2,Vancouver Canucks,3,54.63
3,Florida Panthers,4,54.09
4,Philadelphia Flyers,5,53.04
5,New Jersey Devils,6,52.97
6,Arizona Coyotes,7,52.31
7,Nashville Predators,8,52.28
8,Calgary Flames,9,52.18
9,Buffalo Sabres,10,51.78


In [6]:
gf = df.sort_values(by="GF%", ascending=False, ignore_index=True)
gf.index += 1
gf.reset_index(inplace=True)
gf = gf.rename(columns = {"index":"GF%"})
gf = gf[["Team", "GF%"]]

In [7]:
xgf = df.sort_values(by="xGF%", ascending=False, ignore_index=True)
xgf.index += 1
xgf.reset_index(inplace=True)
xgf = xgf.rename(columns = {"index":"xGF%"})
xgf = xgf[["Team", "xGF%"]]

In [8]:
hdcf = df.sort_values(by="HDCF%", ascending=False, ignore_index=True)
hdcf.index += 1
hdcf.reset_index(inplace=True)
hdcf = hdcf.rename(columns = {"index":"HDCF%"})
hdcf = hdcf[["Team", "HDCF%"]]

In [9]:
sh = df.sort_values(by="SH%", ascending=False, ignore_index=True)
sh.index += 1
sh.reset_index(inplace=True)
sh = sh.rename(columns = {"index":"SH%"})
sh = sh[["Team", "SH%"]]

In [10]:
# merge all DataFrames into one
dfs = [cf, gf, xgf, hdcf, sh]
final_df = reduce(lambda  left,right: pd.merge(left,right,on=['Team'],
                                            how='outer'), dfs)
final_df

,Team,CF%,CF%,GF%,GF%,xGF%,xGF%,HDCF%,HDCF%,SH%,SH%
0,Carolina Hurricanes,1,57.94,2,60.98,2,57.04,2,57.41,22,8.01
1,Edmonton Oilers,2,56.89,6,56.92,1,59.26,1,64.43,8,9.61
2,Vancouver Canucks,3,54.63,19,50.00,7,53.85,5,55.04,11,9.33
3,Florida Panthers,4,54.09,1,64.71,5,54.55,7,54.50,27,7.31
4,Philadelphia Flyers,5,53.04,14,50.98,10,52.76,11,51.85,15,8.61
5,New Jersey Devils,6,52.97,20,49.18,4,54.77,8,52.83,23,7.89
6,Arizona Coyotes,7,52.31,28,42.86,14,51.67,14,50.45,16,8.60
7,Nashville Predators,8,52.28,3,60.42,21,48.87,23,48.15,4,10.32
8,Calgary Flames,9,52.18,8,55.32,13,51.83,22,48.72,7,9.89
9,Buffalo Sabres,10,51.78,9,53.85,19,49.06,12,51.69,28,6.95


In [11]:
# ex. get a team's stats
final_df.loc[final_df["Team"] == "Toronto Maple Leafs"]

,Team,CF%,CF%,GF%,GF%,xGF%,xGF%,HDCF%,HDCF%,SH%,SH%
15,Toronto Maple Leafs,16,50.26,5,58.06,6,54.24,3,55.95,3,10.65


In [12]:
from get_todays_games import getGames

matchups = getGames()
matchups

[('Golden Knights', 'Blue Jackets'),
 ('Panthers', 'Rangers'),
 ('Bruins', 'Maple Leafs'),
 ('Blues', 'Flyers'),
 ('Blackhawks', 'Avalanche'),
 ('Kraken', 'Flames')]

In [13]:
res = pd.DataFrame()

for away, home in matchups:
    away_df = final_df[final_df["Team"].str.contains(away)]
    home_df = final_df[final_df["Team"].str.contains(home)]
    matchup_df = pd.concat([away_df, home_df], ignore_index=True)
    res = pd.concat([res, matchup_df], ignore_index=True)
res


,Team,CF%,CF%,GF%,GF%,xGF%,xGF%,HDCF%,HDCF%,SH%,SH%
0,Vegas Golden Knights,32,42.46,23,46.43,26,46.24,28,43.36,5,10.16
1,Columbus Blue Jackets,19,49.90,10,53.49,9,53.07,21,48.76,14,8.95
2,Florida Panthers,4,54.09,1,64.71,5,54.55,7,54.50,27,7.31
3,New York Rangers,27,44.70,4,58.54,31,40.51,30,40.64,13,9.06
4,Boston Bruins,26,46.34,24,46.15,18,49.14,18,49.61,20,8.28
5,Toronto Maple Leafs,16,50.26,5,58.06,6,54.24,3,55.95,3,10.65
6,St Louis Blues,30,43.69,27,45.00,32,39.45,31,37.88,19,8.33
7,Philadelphia Flyers,5,53.04,14,50.98,10,52.76,11,51.85,15,8.61
8,Chicago Blackhawks,31,43.49,32,31.11,28,43.15,27,43.38,32,5.96
9,Colorado Avalanche,20,49.49,12,52.08,17,49.66,10,52.59,25,7.79


In [14]:
f = open("result.csv", 'w')

for col in res.columns.values:
    f.write(col + ",")

f.write("\n")

i = 0
for col in res.values:
    for row in col:
        f.write(str(row) + ",")
    if i % 2 == 0:
        f.write("\n")
    else:
        f.write("\n\n")
    i += 1

f.close()